In [1]:
# Workflow
# https://techvidvan.com/tutorials/movie-recommendation-system-python-machine-learning/

In [2]:
# imports

import io
import numpy as np
import pandas as pd
import sys
import codecs

import nltk
from gensim.models import Word2Vec, KeyedVectors
from sklearn.manifold import TSNE

%matplotlib inline
import matplotlib.pyplot as plt

import ssl

from ast import literal_eval

In [3]:
# override ssl error from trying to reload nltk data

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

In [4]:
# Load in data
watch_history = pd.read_csv('NetflixViewingHistoryDan.csv')
watch_history

,Title,Date
0,Big Fish,3/28/22
1,Monsters vs. Aliens,3/19/22
2,Brand New Cherry Flavor: Limited Series: Tadpo...,3/10/22
3,Brand New Cherry Flavor: Limited Series: I Exist,3/10/22
4,Starship Troopers,3/9/22
...,...,...
973,Battle for Haditha,3/14/15
974,Blackfish,3/14/15
975,The Immigrant,2/24/15
976,Elsa & Fred,2/23/15


In [5]:
# Netflix database
netflix = pd.read_csv('netflix_titles.csv')
netflix

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
...,...,...,...,...,...,...,...,...,...,...,...,...
8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a..."
8803,s8804,TV Show,Zombie Dumb,NaN,NaN,NaN,"July 1, 2019",2018,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g..."
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."


In [6]:
# Keeping only movie data for watch history
movie_dict = {
    'title': [],
    'director': [],
    'cast': [],
    'list': [],
    'description': []
}

for item in watch_history['Title']:
    match = netflix[netflix['title']== item]
    if match.shape[0] != 0 and 'Movie' in match['type'].tolist():
        movie_dict['title'].append(match['title'].tolist()[0])
        movie_dict['director'].append(match['director'].tolist()[0])
        movie_dict['cast'].append(match['cast'].tolist()[0])
        movie_dict['list'].append(match['listed_in'].tolist()[0])
        movie_dict['description'].append(match['description'].tolist()[0])


In [7]:
# Create a new dataframe of only those titles
movies_df = pd.DataFrame(movie_dict)
movies_df

,title,director,cast,list,description
0,Total Recall,Paul Verhoeven,"Arnold Schwarzenegger, Rachel Ticotin, Sharon ...","Action & Adventure, Sci-Fi & Fantasy","After getting a memory implant, working stiff ..."
1,Someone Great,Jennifer Kaytin Robinson,"Gina Rodriguez, Brittany Snow, DeWanda Wise, L...","Comedies, Romantic Movies","On the heels of a blindsiding breakup, music j..."
2,The Guns of Navarone,J. Lee Thompson,"Gregory Peck, David Niven, Anthony Quinn, Stan...","Action & Adventure, Classic Movies","During World War II, British forces launch an ..."
3,Stripes,Ivan Reitman,"Bill Murray, Harold Ramis, Warren Oates, P.J. ...","Classic Movies, Comedies, Cult Movies","After losing everything, an indolent sad sack ..."
4,Klaus,Sergio Pablos,"Jason Schwartzman, J.K. Simmons, Rashida Jones...","Children & Family Movies, Comedies",A selfish postman and a reclusive toymaker for...
...,...,...,...,...,...
162,The November Man,Roger Donaldson,"Pierce Brosnan, Luke Bracey, Olga Kurylenko, E...",Action & Adventure,An ex-CIA agent emerges from retirement to pro...
163,Kevin Hart: Let Me Explain,"Leslie Small, Tim Story",Kevin Hart,Stand-Up Comedy,Philadelphia funnyman Kevin Hart takes the sta...
164,Bill Burr: Let It Go,Shannon Hartman,Bill Burr,Stand-Up Comedy,The musings of comedian Bill Burr are let loos...
165,Blackfish,Gabriela Cowperthwaite,NaN,Documentaries,This fascinating documentary examines the life...


In [8]:
# Having only movies in our main database
netflix = netflix[netflix['type'] == 'Movie']

In [9]:
def literal_return(val):
    try:
        return literal_eval(val)
    except ValueError:
        return (val)
    except SyntaxError:
        return(val)

In [10]:
# Selecting the features we want to use
features = ["director", "list", "description"]

for feature in features:
    movies_df[feature] = movies_df[feature].apply(literal_return)

movies_df[features].head(10)

,director,list,description
0,Paul Verhoeven,"Action & Adventure, Sci-Fi & Fantasy","After getting a memory implant, working stiff ..."
1,Jennifer Kaytin Robinson,"Comedies, Romantic Movies","On the heels of a blindsiding breakup, music j..."
2,J. Lee Thompson,"Action & Adventure, Classic Movies","During World War II, British forces launch an ..."
3,Ivan Reitman,"Classic Movies, Comedies, Cult Movies","After losing everything, an indolent sad sack ..."
4,Sergio Pablos,"Children & Family Movies, Comedies",A selfish postman and a reclusive toymaker for...
5,Paul Verhoeven,"Action & Adventure, Sci-Fi & Fantasy","After getting a memory implant, working stiff ..."
6,Terry Jones,"Classic Movies, Comedies, Cult Movies","Born in a stable in Judea, Brian grows up to j..."
7,David Gordon Green,"Action & Adventure, Comedies","After witnessing a murder, a perpetually stone..."
8,"Mike Rianda, Jeff Rowe","Children & Family Movies, Comedies",A robot apocalypse put the brakes on their cro...
9,Matt Thompson,"Action & Adventure, Comedies",A chainsaw-wielding George Washington teams wi...


In [11]:
# Clean data for easier conversion to metadata
def clean_data(row):
    if isinstance(row, list):
        return [str.lower(i.replace(" ", "")) for i in row]
    else:
        if isinstance(row, str):
            return str.lower(row.replace(" ", ""))
        else:
            return ""

for feature in features:
    movies_df[feature] = movies_df[feature].apply(clean_data)

In [12]:
# Metadata creation
def create_soup(features):
    return ' '.join(features['director'].split(',')) + ' ' + ' '.join(features['list'].split(',')) + ' ' + ' '.join(features['description'].split(','))


movies_df["soup"] = movies_df.apply(create_soup, axis=1)
print(movies_df["soup"].head())

0    paulverhoeven action&adventure sci-fi&fantasy ...
1    jenniferkaytinrobinson comedies romanticmovies...
2    j.leethompson action&adventure classicmovies d...
3    ivanreitman classicmovies comedies cultmovies ...
4    sergiopablos children&familymovies comedies as...
Name: soup, dtype: object


In [13]:
# Creating one large metadata entry for all watched movies
# Possible replacement with some dimensionality reduction technique
final_soup = ' '.join(movies_df['soup'].tolist())
final_soup

'paulverhoeven action&adventure sci-fi&fantasy aftergettingamemoryimplant workingstiffdouglasquaiddiscovershemightactuallybeasecretagentembroiledinaviolentinsurrectiononmars. jenniferkaytinrobinson comedies romanticmovies ontheheelsofablindsidingbreakup musicjournalistjennybracesforanewbeginning–andonelastadventurewithherclosestfriends. j.leethompson action&adventure classicmovies duringworldwarii britishforceslaunchanattackdesignedtotakeoutthemassivenazicannonsthatguardacriticalseachannel. ivanreitman classicmovies comedies cultmovies afterlosingeverything anindolentsadsackimpulsivelyjoinstheu.s.armyandcajoleshisbestfriendintoenlisting too. sergiopablos children&familymovies comedies aselfishpostmanandareclusivetoymakerformanunlikelyfriendship deliveringjoytoacold darktownthatdesperatelyneedsit. paulverhoeven action&adventure sci-fi&fantasy aftergettingamemoryimplant workingstiffdouglasquaiddiscovershemightactuallybeasecretagentembroiledinaviolentinsurrectiononmars. terryjones classic

In [14]:
# Final database for similarity matrix calculation
movie_database = netflix.drop(['show_id', 'type', 'country', 'date_added', 'release_year', 'rating', 'duration', 'cast'], axis=1)

In [15]:
# Selecting the features we want to use in our final database
features = ["director", "listed_in", "description"]

for feature in features:
    movie_database[feature] = movie_database[feature].apply(literal_return)

movie_database[features].head(10)

,director,listed_in,description
0,Kirsten Johnson,Documentaries,"As her father nears the end of his life, filmm..."
6,"Robert Cullen, José Luis Ucha",Children & Family Movies,Equestria's divided. But a bright-eyed hero be...
7,Haile Gerima,"Dramas, Independent Movies, International Movies","On a photo shoot in Ghana, an American model s..."
9,Theodore Melfi,"Comedies, Dramas",A woman adjusting to life after a loss contend...
12,Christian Schwochow,"Dramas, International Movies",After most of her family is murdered in a terr...
13,Bruno Garotti,"Children & Family Movies, Comedies",When the clever but socially-awkward Tetê join...
16,"Pedro de Echave García, Pablo Azorín Williams","Documentaries, International Movies",Declassified documents reveal the post-WWII li...
18,Adam Salky,Thrillers,After a deadly home invasion at a couple’s new...
22,K.S. Ravikumar,"Comedies, International Movies",Newly divorced and denied visitation rights wi...
23,"Alex Woo, Stanley Moore",Children & Family Movies,From arcade games to sled days and hiccup cure...


In [16]:
# Clean data for easier conversion to metadata
def clean_data(row):
    if isinstance(row, list):
        return [str.lower(i.replace(" ", "")) for i in row]
    else:
        if isinstance(row, str):
            return str.lower(row.replace(" ", ""))
        else:
            return ""

for feature in features:
    movie_database[feature] = movie_database[feature].apply(clean_data)

In [17]:
# Metadata creation
def create_soup(features):
    return ' '.join(features['director'].split(',')) + ' ' + ' '.join(features['listed_in'].split(',')) + ' ' + ' '.join(features['description'].split(','))


movie_database["soup"] = movie_database.apply(create_soup, axis=1)
print(movie_database["soup"].head())

0     kirstenjohnson documentaries asherfathernearst...
6     robertcullen joséluisucha children&familymovie...
7     hailegerima dramas independentmovies internati...
9     theodoremelfi comedies dramas awomanadjustingt...
12    christianschwochow dramas internationalmovies ...
Name: soup, dtype: object


In [18]:
movie_database

,title,director,listed_in,description,soup
0,Dick Johnson Is Dead,kirstenjohnson,documentaries,"asherfathernearstheendofhislife,filmmakerkirst...",kirstenjohnson documentaries asherfathernearst...
6,My Little Pony: A New Generation,"robertcullen,joséluisucha",children&familymovies,equestria'sdivided.butabright-eyedherobelieves...,robertcullen joséluisucha children&familymovie...
7,Sankofa,hailegerima,"dramas,independentmovies,internationalmovies","onaphotoshootinghana,anamericanmodelslipsbacki...",hailegerima dramas independentmovies internati...
9,The Starling,theodoremelfi,"comedies,dramas",awomanadjustingtolifeafteralosscontendswithafe...,theodoremelfi comedies dramas awomanadjustingt...
12,Je Suis Karl,christianschwochow,"dramas,internationalmovies",aftermostofherfamilyismurderedinaterroristbomb...,christianschwochow dramas internationalmovies ...
...,...,...,...,...,...
8801,Zinzana,majidalansari,"dramas,internationalmovies,thrillers",recoveringalcoholictalalwakesupinsideasmall-to...,majidalansari dramas internationalmovies thril...
8802,Zodiac,davidfincher,"cultmovies,dramas,thrillers","apoliticalcartoonist,acrimereporterandapairofc...",davidfincher cultmovies dramas thrillers apoli...
8804,Zombieland,rubenfleischer,"comedies,horrormovies","lookingtosurviveinaworldtakenoverbyzombies,ado...",rubenfleischer comedies horrormovies lookingto...
8805,Zoom,peterhewitt,"children&familymovies,comedies","draggedfromcivilianlife,aformersuperheromusttr...",peterhewitt children&familymovies comedies dra...


In [19]:
title = ' '.join(movies_df['title'].tolist())
directors= ' '.join(movies_df['title'].tolist())
listed_in = ' '.join(movies_df['list'].tolist())
description = ' '.join(movies_df['description'].tolist())

In [20]:
movie_database.loc[8807] = [title, directors, listed_in,description, final_soup]

In [28]:
movie_database['soup'][2]

'hailegerima dramas independentmovies internationalmovies onaphotoshootinghana anamericanmodelslipsbackintime becomesenslavedonaplantationandbearswitnesstotheagonyofherancestralpast.'

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

count_vectorizer = CountVectorizer(stop_words="english")
count_matrix = count_vectorizer.fit_transform(movie_database["soup"])

print(count_matrix.shape)

cosine_sim2 = cosine_similarity(count_matrix, count_matrix) 
print(cosine_sim2.shape)

movie_database = movie_database.reset_index()
indices = pd.Series(movie_database.index, index=movie_database['title'])

(6132, 22460)
(6132, 6132)


In [23]:
indices = pd.Series(movie_database.index, index=movie_database["title"]).drop_duplicates()

print(indices.head())

title
Dick Johnson Is Dead                0
My Little Pony: A New Generation    1
Sankofa                             2
The Starling                        3
Je Suis Karl                        4
dtype: int64


In [24]:
def get_recommendations(title, cosine_sim=cosine_sim2):
    idx = indices[title]
    similarity_scores = list(enumerate(cosine_sim[idx]))
    similarity_scores= sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores= similarity_scores[1:11]
    # (a, b) where a is id of movie, b is similarity_scores

    movies_indices = [ind[0] for ind in similarity_scores]
    movies = movie_database["title"].iloc[movies_indices]
    return movies

In [25]:
print("################ Content Based System #############")
# print("Recommendations for The Dark Knight Rises")
# print(get_recommendations("The Dark Knight Rises", cosine_sim2))
# print()
print("Recommendations for Netflix Home")
print(get_recommendations(indices.index[-1], cosine_sim2))

################ Content Based System #############
Recommendations for Netflix Home
2480                  Shortcut Safari
3769                      The Do-Over
5474                  Sherlock Holmes
228                The Last Boy Scout
514                   Starsky & Hutch
617              The Whole Nine Yards
3192                  Game Over, Man!
5830                   The Other Guys
6119    You Don't Mess with the Zohan
1246              Free State of Jones
Name: title, dtype: object


In [26]:
print("################ Content Based System #############")
# print("Recommendations for The Dark Knight Rises")
# print(get_recommendations("The Dark Knight Rises", cosine_sim2))
# print()
print("Recommendations for My Little Pony")
print(get_recommendations('My Little Pony: A New Generation', cosine_sim2))

################ Content Based System #############
Recommendations for My Little Pony
2258                               True: Tricky Treat Day
277                     Chhota Bheem: The Rise of Kirmada
1697        Thomas & Friends: Thomas and the Royal Engine
1781                Chhota Bheem and the Curse of Damyaan
1842                             Steam Team to the Rescue
2149                 Escape from Mr. Lemoncello’s Library
2195       ChuChuTV Surprise Eggs Learning Videos (Hindi)
2235                           Little Baby Bum: Go Buster
2306                        Super Monsters Back to School
2623    LEGO Marvel Super Heroes: Guardians of the Galaxy
Name: title, dtype: object
